In [113]:
include("../../../Ofdm.jl")
include("../../../../../Channel/Channel.jl")
using Plots, LinearAlgebra, FFTW

In [134]:
# 変調パラメータ
nbits = 2000
M = 4
qam = Ofdm.QamMod(M)
nfft=64; cpsize=16; npi=16; ngc=0; ndim=(2,2)
params = Ofdm.OfdmParams(nfft,cpsize,npi,ngc,ndim; pilot_type=:comb, pilot_space=4, pilot_interval=2)
indices = Ofdm.get_indices(params)
tx_frame = Ofdm.gen_frame(params, indices, 20);

In [143]:
# 通信路パラメータ
N0 = 10^(-30/10)
pdp = Channel.exponent(10,20);

In [144]:
# 送信機
tx_bit = rand(nbits) .> 0.5
tx_symbol = Ofdm.qammod(qam, tx_bit)
tx_ofdmsig = Ofdm.ofdmmod(params, tx_frame, tx_symbol, indices);

In [145]:
# 通信路
rx_ofdmsig, cir = Channel.multipath_fading(tx_ofdmsig, ndim, pdp, istail=false)
rx_ofdmsig = Channel.awgn!(rx_ofdmsig, N0);
# 受信機
rx_frame = Ofdm.ofdmdemod(params, rx_ofdmsig);

In [146]:
# チャネル推定
function chest(params, rx_frame, tx_frame, indices)
    cfr_est = zeros(ComplexF64, size(rx_frame)..., size(tx_frame,3)); # チャネル推定配列
    Ofdm.pilot_divide!(cfr_est, rx_frame, tx_frame, indices[:frame])
    # Ofdm.interpolate!(cfr_est, :time, indices[:pilot], indices[:frame])
    # Ofdm.interpolate!(cfr_est, :freq, indices[:pilot], indices[:frame])
    ave_cfr = Ofdm.time_ave_pilot(cfr_est, indices[:pilot], indices[:frame])
    F = fft!(Array{ComplexF64}(I, params.nfft, params.nfft),1)
    pind = sort(vec(indices[:pilot]))
    cir_est = zeros(eltype(ave_cfr), params.cpsize, size(ave_cfr,2), size(ave_cfr,3))
    for n in axes(ave_cfr,2)
        for m in axes(ave_cfr,3)
            cir_est[:,m,n] = Ofdm.cir_est(view(ave_cfr,:,m,n), F, pind, 1:params.cpsize, method=:LS)
        end
    end
    return Ofdm.get_cfr(cir_est, params.nfft, size(rx_frame,2))
end

chest (generic function with 1 method)

In [147]:
cfr_est = chest(params, rx_frame, tx_frame, indices);

In [150]:
cfr_est = chest(params, rx_frame, tx_frame, indices);
rx_symbol = Ofdm.equalize(rx_frame, cfr_est, indices[:frame], N0,method=:MMSE)

1000-element Array{Complex{Float64},1}:
  0.7164924318151754 - 0.7670406732481917im
  0.7099390136846984 - 0.7156903562443278im
 -0.7384450604735814 + 0.7187836223280406im
  0.6978134163333914 + 0.6830652532516238im
 -0.7094079905083225 - 0.6921169398662447im
  0.6978231051591794 + 0.7090843787028213im
 -0.7102868687025231 - 0.6756324507438708im
 -0.6845792826757122 + 0.7154847761865599im
  -0.675881126295887 - 0.7056964316611545im
  0.6713588002209978 + 0.7014071453828643im
 -0.6998039072571084 - 0.7326689187297604im
  0.6184251418651441 + 0.7708874705745897im
 -0.7086115094643592 - 0.6770518961907037im
                     ⋮                     
  0.7658286345691587 - 0.6994880483731013im
 -0.7275484713146811 + 0.7156063783966861im
 -0.7454350154670395 + 0.7280806501842829im
 -0.7327983800799792 - 0.7036245537139494im
  0.7425416155437516 + 0.6352236619476539im
  0.7368004964601931 + 0.7039817796768824im
 -0.7067913721526119 + 0.7204444704463691im
 -0.7051132190990932 + 0.72157939114

In [133]:
rx_bit = Ofdm.qamdemod(qam, rx_symbol);
sum(tx_bit .!== rx_bit)

81